<a href="https://colab.research.google.com/github/mazabdul7/AtTheEdge/blob/main/New_Framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pathlib
from tensorflow.keras import layers
import zipfile as zf

In [4]:
files = zf.ZipFile("dataYoupengSplit.zip",'r')
files.extractall()
files.close()

In [ ]:
#pip install -q tensorflow-model-optimization

     |████████████████████████████████| 174kB 13.9MB/s 


In [ ]:
print(tf.__version__) # TF Version

Generates a data object with the dataset. Splits 80/20 test/val

In [ ]:
batch_size = 16
img_height = 288
img_width = 384

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  r'/content/main',
  validation_split=0.2,
  subset="training",
  seed=777,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  r'/content/main',
  validation_split=0.2,
  subset="validation",
  seed=777,
  image_size=(img_height, img_width),
  batch_size=batch_size)

#Print number of samples
print("Number of training samples: %d" % tf.data.experimental.cardinality(train_ds))
print(
    "Number of validation samples: %d" % tf.data.experimental.cardinality(val_ds)
)

print("The labels for the ds: %s" % train_ds.class_names)

In [ ]:
#Visualise the inputs
class_names = train_ds.class_names

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
#Set cache and prefetch for drive disk performance
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)

In [7]:
#Augment training data to reduce overfitting by creating a pre-proccessing layer that only activates in training

data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"), #random hor flip
        layers.experimental.preprocessing.RandomFlip("vertical"), # random ver flip
        layers.experimental.preprocessing.RandomRotation(0.5), #random rotation up to 180deg
        layers.experimental.preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1, fill_mode='reflect'), #translation
        layers.experimental.preprocessing.RandomZoom((0, -1), (0, -1)), #up to 100% zoom
    ]
)


In [ ]:
#Visualise transformations on a sample image 
for images, labels in train_ds.take(1):
    plt.figure(figsize=(10, 10))
    first_image = images[4]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(
            tf.expand_dims(first_image, 0), training=True
        )
        plt.imshow(augmented_image[0].numpy().astype("int32"))
        plt.title(int(labels[i]))
        plt.axis("off")

Begin building out models!

In [ ]:
#Build our model

base_model = keras.applications.EfficientNetB0(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(img_height, img_width, 3),
    include_top=False,
)

In [ ]:
#View the layers in the base model
#May want to avoid models with advanced layering e.g squeeze, unsqueeze... as they may cause issues when utilising TensorRT later. There are workarounds though.
base_model.summary()

In [9]:
# Unfreeze base model
base_model.trainable = True

# Create new model on top
inputs = keras.Input(shape=( img_height, img_width, 3))
x = data_augmentation(inputs)  # Apply random data augmentation

In [10]:
# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
#x = keras.layers.BatchNormalization()(x)
#x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(5, activation='softmax')(x)
model = keras.Model(inputs, outputs,)

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 288, 384, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 288, 384, 3)       0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 9, 12, 1280)       4049571   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 6405      
Total params: 4,055,976
Trainable params: 4,013,953
Non-trainable params: 42,023
_________________________________________________________________


In [14]:
#Train model with low LR as suggested in the report
model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(4.3e-5),  # Low learning rate identified by lowest loss within the first 5 epochs
    loss="sparse_categorical_crossentropy" ,
    metrics=["accuracy"],
)

epochs = 33
model.fit(train_ds, epochs=epochs, validation_data=val_ds)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 288, 384, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 288, 384, 3)       0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 9, 12, 1280)       4049571   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 6405      
Total params: 4,055,976
Trainable params: 4,013,953
Non-trainable params: 42,023
_________________________________________________________________
Epoch 1/10
108/108 [==============================] - 45s 421ms/step - loss: 0.2060 - accuracy: 0.9245 - 

In [18]:
#Retrain at a lower learning rate for stability
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=4e-6),
    loss="sparse_categorical_crossentropy" ,
    metrics=["accuracy"],
)

epochs = 5
model.fit(train_ds, epochs=epochs, validation_data=val_ds)

Epoch 1/5
108/108 [==============================] - 45s 416ms/step - loss: 0.1400 - accuracy: 0.9472 - val_loss: 0.1902 - val_accuracy: 0.9465
Epoch 2/5
108/108 [==============================] - 43s 398ms/step - loss: 0.1656 - accuracy: 0.9373 - val_loss: 0.1861 - val_accuracy: 0.9535
Epoch 3/5
108/108 [==============================] - 43s 401ms/step - loss: 0.1534 - accuracy: 0.9448 - val_loss: 0.1853 - val_accuracy: 0.9512
Epoch 4/5
108/108 [==============================] - 43s 399ms/step - loss: 0.1457 - accuracy: 0.9454 - val_loss: 0.1887 - val_accuracy: 0.9442
Epoch 5/5
108/108 [==============================] - 43s 401ms/step - loss: 0.1296 - accuracy: 0.9512 - val_loss: 0.1792 - val_accuracy: 0.9558


IMPORT EXPORT

In [20]:
#Save whole model
model.save('EfficientNetB0Small2.h5', include_optimizer=False) 


In [ ]:
#load model
model = tf.keras.models.load_model('EfficientNetB0.75', compile=False)

TEST BENCH

In [ ]:
files = zf.ZipFile("B0.zip",'r')
files.extractall()
files.close()

In [12]:
#Import test set as images
img = keras.preprocessing.image.DirectoryIterator(
    r'/content/test', tf.keras.preprocessing.image.ImageDataGenerator(), target_size=(img_height, img_width), batch_size=1)

Found 238 images belonging to 5 classes.


In [19]:
#Calculate accuracy against test set
labels = ["cardboard", "glass", "metal", "paper", "plastic"]
acc = 0
for i in range(len(img)):
  predictions = model.predict(img[i])
  score = tf.nn.softmax(predictions[0])
  for j in img[i][1]:
    if np.argmax(score) == j.tolist().index(1):
      acc += 1
    #Prints confidence of each inference
    #print("%s:%s %.2f" % (labels[np.argmax(score)], labels[j.tolist().index(1)], predictions[0][np.argmax(score)]*100))
print("The accuracy against the test set was found to be: " + str((acc/len(img))*100) + "%")


The accuracy against the test set was found to be: 95.37815126050421%


In [ ]:
#TFLite Converter (No longer used for project)
converter = tf.lite.TFLiteConverter.from_keras_model(model2)
tflite_model = converter.convert()

with open('B0TFlite2.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpmputmep6/assets


INFO:tensorflow:Assets written to: /tmp/tmpmputmep6/assets


TRT ENGINE OPTIMISATIONS

**These MUST be carried out on the device that the model will be run on as optimisations are made specifically for the available GPUS**

In [ ]:
pip install --upgrade tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.python.compiler.tensorrt import trt_convert as trt
#set engine parameters
conversion_params = tf.experimental.tensorrt.ConversionParams(max_workspace_size_bytes=1<<25, precision_mode='FP16', max_batch_size = 1)

#int8 gen
def input_fn():
    for i in range(10):
        yield tf.random.normal((1, 512, 384, 3))
#convert the model to a graph
converter = trt.TrtGraphConverterV2(
    input_saved_model_dir='EfficientNetB0',
    conversion_params=conversion_params)

converter.convert() # convert subgraphs

#providing sample inputs here allows engines to be created ahead of time which saves costs at inference (removes overheads)
def my_input_fn():
# Input for a single inference call, for a network that has two input tensors:
  inp1 = img[0][0]
  inp2 = img[1][0]
  yield (inp1)

#build and save model
converter.build(input_fn=my_input_fn)
converter.save('TRTModel')

In [ ]:
#load optimised graph
from tensorflow.python.compiler.tensorrt import trt_convert as trt
from tensorflow.python.saved_model import signature_constants, tag_constants
from tensorflow.python.framework import convert_to_constants
import time
saved_model_loaded = tf.saved_model.load(
    'content/new3', tags=[tag_constants.SERVING])

graph_func = saved_model_loaded.signatures[signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY]

In [ ]:
#test accuracy
acc = 0
for i in range(len(img)):
  x = tf.constant(img[i][0])
  predictions = graph_func(x)
  score = tf.nn.softmax(predictions.get('dense').numpy()[0])
  for j in img[i][1]:
    if np.argmax(score) == j.tolist().index(1):
      acc += 1
print("The accuracy against the test set was found to be: " + str((acc/len(img))*100) + "%")

The accuracy against the test set was found to be: 95.37815126050421%


In [ ]:
#tf.constant(np.expand_dims(img[1][0], axis = 0))

In [ ]:
#img = tf.keras.preprocessing.image.load_img('/content/test/cardboard/cardboard101.jpg', target_size=(512, 384))
#x = tf.keras.preprocessing.image.img_to_array(img)
#x = np.array([x])

In [ ]:
x.shape

In [7]:
#Zip savedmodels and export for download
!zip -r /content/B075.zip /content/EfficientNetB0.75

updating: content/EfficientNetB0.75/ (stored 0%)
updating: content/EfficientNetB0.75/variables/ (stored 0%)
updating: content/EfficientNetB0.75/variables/variables.index (deflated 78%)
updating: content/EfficientNetB0.75/variables/variables.data-00000-of-00001 (deflated 7%)
updating: content/EfficientNetB0.75/assets/ (stored 0%)
updating: content/EfficientNetB0.75/saved_model.pb (deflated 92%)
